In [1]:
from pathlib import Path
import os

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn.pipeline import Pipeline

from data_processing import *
# hacky shit
# from data_processing.data_processing import *

In [2]:
MAIN_PATH = Path(os.getcwd()).parent

## Load data

In [3]:
org_df = pd.read_csv(os.path.join(MAIN_PATH, "data", "data.csv"))

# copied from load() function but modified to aggregate data and not drop it

org_df['czas'] = pd.to_datetime(org_df['czas'], utc=True)
org_df.set_index('czas', inplace=True)


temp_zuz = pd.read_csv(os.path.join(MAIN_PATH, "data", "temp_zuz_fixed.csv"))


temp_zuz['Czas'] = pd.to_datetime(temp_zuz['Czas'], utc=True)
temp_zuz.set_index('Czas', inplace=True)
temp_zuz = temp_zuz.rename(columns={'Czas': 'czas'})

org_df.head(3)

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf01.daca.pv,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 22:00:00+00:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.090454,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.6,4.55,9.87
2020-09-30 22:01:00+00:00,54.771942,54.734675,12.026410,22.593412,108.846612,112.334600,109.906645,108.879083,110.644046,104.371977,...,92.090534,92.024206,92.405239,24.663982,24.526161,4.48,8.98,8.6,4.55,9.87
2020-09-30 22:02:00+00:00,54.695816,54.774163,11.953671,22.428933,108.827640,112.362508,109.970932,108.844765,110.621665,104.387912,...,92.090613,92.024334,92.405196,24.662656,24.526161,4.48,8.98,8.6,4.55,9.87


## Simple features based on correlation

In [4]:
df_cleared = pd.DataFrame(index=org_df.index)

In [5]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = org_df.columns[org_df.columns.str.contains('001tix')].values
df_cleared['TIX1'] = org_df[TIX1].mean(axis='columns')

In [6]:
# srednia po 'WODY df KOLEKTORÓW [°C]'
TIR = org_df.columns[org_df.columns.str.contains('tir')].values
df_cleared['TIR'] = org_df[TIR].mean(axis='columns')

In [7]:
# dodanie reg nadawy koncentratu
FCX = org_df.columns[org_df.columns.str.contains('fcx')].values
df_cleared[FCX] = org_df[FCX]

In [8]:
# sumaryczna moc cieplna
NIR = org_df.columns[org_df.columns.str.contains('nir')].values
df_cleared[NIR] = org_df[NIR]

In [9]:
df_cleared.shape

(540492, 7)

### Aggregation and lagging

In [10]:
# Renaming columns

feature_desc_df = pd.read_csv(os.path.join(MAIN_PATH, "data_processing", "feature_desc.csv"), index_col='name')

def feature_desc(name):
        try:
            if type(name)==str:
                return feature_desc_df.loc[name]['desc']
            else:
                return feature_desc_df.loc[name]['desc'].values
        except:
            return name


col_names = list(df_cleared.columns.values)

for col_name in col_names:
    df_cleared = df_cleared.rename(columns={col_name: feature_desc(col_name)})

In [11]:
df_cleared.columns

Index(['TIX1', 'TIR', 'REG NADAWY KONCENTRATU LIW1 [Mg/h]',
       'REG NADAWY KONCENTRATU LIW2 [Mg/h]',
       'REG KONCENTRAT PRAZONY LIW3 [Mg/h]', 'REG PYL ZWROT LIW4 [Mg/h]',
       'SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]'],
      dtype='object')

In [13]:
# Aggregating
template_df = df_cleared.copy()
AGGREGATE_INTERVAL = 20

agg_df = aggregate(df_cleared, AGGREGATE_INTERVAL)
agg_df.head(3)

/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]
czas,,,,,,,
2020-09-30 22:00:00+00:00,418.987794,29.090193,55.192327,54.903059,12.011326,22.505063,14.384833
2020-09-30 22:15:00+00:00,418.976831,29.054909,54.760062,54.936170,11.999173,22.452618,14.551195
2020-09-30 22:30:00+00:00,418.956943,29.013282,54.903154,55.049953,12.007457,22.450732,14.731094


In [14]:
agg_df.shape

(36034, 7)

In [15]:
# Adding lagged featues

lagged_df = agg_df.copy()

for i in range(60 // AGGREGATE_INTERVAL):
    lagged_df = pd.concat([lagged_df, agg_df.shift(i+1).add_prefix(f'{i+1}th_agg_')], axis=1)

# delete rows with nans created by shifts
lagged_df = lagged_df.iloc[(60 // AGGREGATE_INTERVAL):, :]
    
lagged_df.head(3)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,3th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],3th_agg_REG PYL ZWROT LIW4 [Mg/h],3th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_TIX1,4th_agg_TIR,4th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],4th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],4th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],4th_agg_REG PYL ZWROT LIW4 [Mg/h],4th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 23:00:00+00:00,418.911529,28.995352,54.880940,54.947519,11.524826,22.568504,14.977978,418.934559,28.975699,54.765774,...,11.999173,22.452618,14.551195,418.987794,29.090193,55.192327,54.903059,12.011326,22.505063,14.384833
2020-09-30 23:15:00+00:00,418.888202,28.975571,54.811015,54.985904,10.984112,22.520578,15.063879,418.911529,28.995352,54.880940,...,12.007457,22.450732,14.731094,418.976831,29.054909,54.760062,54.936170,11.999173,22.452618,14.551195
2020-09-30 23:30:00+00:00,418.864898,28.954360,54.768437,54.978093,11.000078,22.467616,15.083688,418.888202,28.975571,54.811015,...,11.997661,22.534326,14.879700,418.956943,29.013282,54.903154,55.049953,12.007457,22.450732,14.731094


In [16]:
lagged_df.shape

(36030, 35)

In [17]:
temp_zuz.head(5)

,temp_zuz
Czas,
2020-09-30 22:00:00+00:00,1297
2020-09-30 23:00:00+00:00,1295
2020-10-01 00:00:00+00:00,1303
2020-10-01 00:30:00+00:00,1302
2020-10-01 01:00:00+00:00,1303


In [18]:
lagged_df.head(5)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,3th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],3th_agg_REG PYL ZWROT LIW4 [Mg/h],3th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_TIX1,4th_agg_TIR,4th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],4th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],4th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],4th_agg_REG PYL ZWROT LIW4 [Mg/h],4th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 23:00:00+00:00,418.911529,28.995352,54.880940,54.947519,11.524826,22.568504,14.977978,418.934559,28.975699,54.765774,...,11.999173,22.452618,14.551195,418.987794,29.090193,55.192327,54.903059,12.011326,22.505063,14.384833
2020-09-30 23:15:00+00:00,418.888202,28.975571,54.811015,54.985904,10.984112,22.520578,15.063879,418.911529,28.995352,54.880940,...,12.007457,22.450732,14.731094,418.976831,29.054909,54.760062,54.936170,11.999173,22.452618,14.551195
2020-09-30 23:30:00+00:00,418.864898,28.954360,54.768437,54.978093,11.000078,22.467616,15.083688,418.888202,28.975571,54.811015,...,11.997661,22.534326,14.879700,418.956943,29.013282,54.903154,55.049953,12.007457,22.450732,14.731094
2020-09-30 23:45:00+00:00,418.842335,28.955376,54.802648,54.969232,10.999920,20.656767,15.131360,418.864898,28.954360,54.768437,...,11.524826,22.568504,14.977978,418.934559,28.975699,54.765774,54.987262,11.997661,22.534326,14.879700
2020-10-01 00:00:00+00:00,418.820060,29.012449,54.768672,54.991230,10.995348,19.991612,15.191047,418.842335,28.955376,54.802648,...,10.984112,22.520578,15.063879,418.911529,28.995352,54.880940,54.947519,11.524826,22.568504,14.977978


In [19]:
lagged_df = lagged_df.join(temp_zuz['temp_zuz'])
lagged_df = lagged_df.dropna(axis=0)

# dodanie temperatury żużla
lagged_df = lagged_df.rename(columns={'temp_zuz': 'TEMP_ZUZ'})

In [20]:
df = lagged_df.copy()

In [21]:
df.head(25)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,3th_agg_REG PYL ZWROT LIW4 [Mg/h],3th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_TIX1,4th_agg_TIR,4th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],4th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],4th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],4th_agg_REG PYL ZWROT LIW4 [Mg/h],4th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],TEMP_ZUZ
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 23:00:00+00:00,418.911529,28.995352,54.880940,54.947519,11.524826,22.568504,14.977978,418.934559,28.975699,54.765774,...,22.452618,14.551195,418.987794,29.090193,55.192327,54.903059,12.011326,22.505063,14.384833,1295.0
2020-10-01 00:00:00+00:00,418.820060,29.012449,54.768672,54.991230,10.995348,19.991612,15.191047,418.842335,28.955376,54.802648,...,22.520578,15.063879,418.911529,28.995352,54.880940,54.947519,11.524826,22.568504,14.977978,1303.0
2020-10-01 00:30:00+00:00,418.777271,29.058571,54.908362,54.972407,11.000747,20.080419,14.847301,418.798185,29.037174,54.712681,...,20.656767,15.131360,418.864898,28.954360,54.768437,54.978093,11.000078,22.467616,15.083688,1302.0
2020-10-01 01:00:00+00:00,418.735368,29.083732,54.874806,55.015018,10.993126,20.995138,14.614946,418.756421,29.065615,54.894533,...,19.974205,15.097498,418.820060,29.012449,54.768672,54.991230,10.995348,19.991612,15.191047,1303.0
2020-10-01 02:00:00+00:00,418.649147,29.083728,54.833627,54.969008,13.001509,20.905994,14.237699,418.671552,29.087527,54.863801,...,21.040453,14.600170,418.735368,29.083732,54.874806,55.015018,10.993126,20.995138,14.614946,1306.0
2020-10-01 03:00:00+00:00,418.568565,28.936682,55.034585,54.984403,12.991980,23.019244,13.721707,418.586988,28.955338,54.807467,...,22.584225,14.092764,418.649147,29.083728,54.833627,54.969008,13.001509,20.905994,14.237699,1303.0
2020-10-01 04:00:00+00:00,418.496677,28.889406,54.788706,54.996404,12.984104,22.986577,13.822403,418.513813,28.866258,54.961041,...,23.008658,13.644978,418.568565,28.936682,55.034585,54.984403,12.991980,23.019244,13.721707,1304.0
2020-10-01 05:00:00+00:00,418.446714,29.006014,54.811576,54.984958,13.001419,23.704587,14.522728,418.457122,28.880973,54.724974,...,22.992760,13.964081,418.496677,28.889406,54.788706,54.996404,12.984104,22.986577,13.822403,1302.0
2020-10-01 06:00:00+00:00,418.405873,29.152039,54.986767,54.972312,13.018621,23.955907,15.045756,418.415919,29.074250,54.876391,...,23.994219,14.554517,418.446714,29.006014,54.811576,54.984958,13.001419,23.704587,14.522728,1303.0


## Split data

In [128]:
train, val, test = split(df)

In [129]:
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

In [130]:
ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model.predict(X_val))

34.447287936027294

## Most of the code above is ignored and moved to generate_dataset function that takes interval as paramter

# Testing different models

## Ridge regression

In [25]:
def generate_dataset(arg_df, interval, temp_zuz):
    
    internal_df = aggregate(arg_df, interval)

    internal_final_df = internal_df.copy(deep=True)
    for i in range(60 // interval):
        internal_final_df = pd.concat([internal_final_df, internal_df.shift(i+1).add_prefix(f'{i+1}th_agg_')], axis=1)

    # delete rows with nans created by shifts
    internal_final_df = internal_final_df.iloc[(60 // interval):, :]


    internal_final_df = internal_final_df.join(temp_zuz['temp_zuz'])
    internal_final_df = internal_final_df.dropna(axis=0)
    # dodanie temperatury żużla
    internal_final_df = internal_final_df.rename(columns={'temp_zuz': 'TEMP_ZUZ'})

    return internal_final_df

In [26]:
solvers = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
alphas = [.1, .5, 1]
# solvers = ['svd']
# alphas [1]

for interval in [5, 10, 15, 20, 30]:
    # creating dataset
    df = generate_dataset(template_df, interval, temp_zuz)
    train, val, test = split(df)
    X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
    y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]
    print(f'------------------ INTERVAL: {interval}min --------------------')
    for solver in solvers:
        for alpha in alphas:
            ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                                    ('ridge', linear_model.Ridge(alpha=alpha, solver=solver))])
            ridge_model.fit(X_train, y_train)
            mse = metrics.mean_squared_error(y_val, ridge_model.predict(X_val))
            print(interval, solver, alpha, mse)

/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


------------------ INTERVAL: 5min --------------------
5 svd 0.1 37.38240095981045
5 svd 0.5 37.297017795267884
5 svd 1 37.225111585294506
5 cholesky 0.1 37.3824009598119
5 cholesky 0.5 37.29701779526787
5 cholesky 1 37.22511158529475
5 lsqr 0.1 36.82748105570309
5 lsqr 0.5 36.82624546862072
5 lsqr 1 36.82486566211473
5 sparse_cg 0.1 37.24637298540301
5 sparse_cg 0.5 37.21122903444472
5 sparse_cg 1 37.166175737042714
5 sag 0.1 37.038360463478384
5 sag 0.5 37.024191013700765
5 sag 1 37.00732382872674
5 saga 0.1 36.94510616886513
5 saga 0.5 36.9373379056318
5 saga 1 36.92900600527612
------------------ INTERVAL: 10min --------------------
10 svd 0.1 35.93778054156256
10 svd 0.5 35.9189407518609


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


10 svd 1 35.895133948686365
10 cholesky 0.1 35.93778054156305
10 cholesky 0.5 35.918940751861044
10 cholesky 1 35.89513394868644
10 lsqr 0.1 35.74936336681257
10 lsqr 0.5 35.754055216858895
10 lsqr 1 35.76000781828741
10 sparse_cg 0.1 35.89246452682019
10 sparse_cg 0.5 35.87909651351979
10 sparse_cg 1 35.86536381839284
10 sag 0.1 35.85817893138123
10 sag 0.5 35.84966608869559
10 sag 1 35.84252346169569
10 saga 0.1 35.82827083485321
10 saga 0.5 35.82588366799285
10 saga 1 35.82544240560554
------------------ INTERVAL: 15min --------------------
15 svd 0.1 34.29288944646824
15 svd 0.5 34.28176009972279
15 svd 1 34.26500210123045
15 cholesky 0.1 34.292889446468024
15 cholesky 0.5 34.281760099722625


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


15 cholesky 1 34.265002101230365
15 lsqr 0.1 34.185045353469015
15 lsqr 0.5 34.18611124823964
15 lsqr 1 34.18820910115524
15 sparse_cg 0.1 34.31697816802682
15 sparse_cg 0.5 34.292548225512796
15 sparse_cg 1 34.273243916872715
15 sag 0.1 34.26756432731755
15 sag 0.5 34.258135144474274
15 sag 1 34.250149971102665
15 saga 0.1 34.248948241045646
15 saga 0.5 34.246102086237414
15 saga 1 34.244355669779964
------------------ INTERVAL: 20min --------------------
20 svd 0.1 33.96898499792789
20 svd 0.5 33.954514120752066
20 svd 1 33.940969936432836
20 cholesky 0.1 33.96898499792782
20 cholesky 0.5 33.95451412075209
20 cholesky 1 33.940969936432865


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


20 lsqr 0.1 33.98523078038912
20 lsqr 0.5 33.98916777214773
20 lsqr 1 33.994070544936996
20 sparse_cg 0.1 33.9599656156399
20 sparse_cg 0.5 33.94738166027458
20 sparse_cg 1 33.93811564344651
20 sag 0.1 33.92841454292137
20 sag 0.5 33.92231670663176
20 sag 1 33.91781128891973
20 saga 0.1 33.9064995154326
20 saga 0.5 33.905530500123064
20 saga 1 33.9064904207464
------------------ INTERVAL: 30min --------------------
30 svd 0.1 34.46989633920034
30 svd 0.5 34.457337874087365
30 svd 1 34.447287936027294


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


30 cholesky 0.1 34.46989633920025
30 cholesky 0.5 34.45733787408731
30 cholesky 1 34.447287936027294
30 lsqr 0.1 34.70204869201267
30 lsqr 0.5 34.70486997663178
30 lsqr 1 34.70840138585983
30 sparse_cg 0.1 34.471527560129964
30 sparse_cg 0.5 34.45546093034332
30 sparse_cg 1 34.44509848180842
30 sag 0.1 34.442224734480966
30 sag 0.5 34.437672852549305
30 sag 1 34.43688538848267
30 saga 0.1 34.42854287538264
30 saga 0.5 34.4304393820431
30 saga 1 34.43485950115067


# Lagged but without aggregation
Lagged features from N minutes before measuremnet of temperature

In [110]:
temp_zuz['temp_zuz']

Czas
2020-09-30 22:00:00+00:00    1297
2020-09-30 23:00:00+00:00    1295
2020-10-01 00:00:00+00:00    1303
2020-10-01 00:30:00+00:00    1302
2020-10-01 01:00:00+00:00    1303
                             ... 
2022-01-31 19:00:00+00:00    1305
2022-01-31 20:00:00+00:00    1304
2022-01-31 21:00:00+00:00    1308
2022-01-31 22:00:00+00:00    1309
2022-01-31 23:00:00+00:00    1305
Name: temp_zuz, Length: 11384, dtype: int64

In [147]:
MINUTES_BEFORE = 10

df_lagged_no_agg = template_df.copy()

df_lagged_no_agg.reset_index(inplace=True)
mask = (df_lagged_no_agg['czas'].dt.minute == 0) | (df_lagged_no_agg['czas'].dt.minute > 60 - MINUTES_BEFORE)

df_lagged_no_agg = df_lagged_no_agg.loc[mask]
df_lagged_no_agg.set_index('czas', inplace=True)
                   
final_df = df_lagged_no_agg.copy(deep=True)
for i in range(MINUTES_BEFORE):
    final_df = pd.concat([final_df, df_lagged_no_agg.shift(MINUTES_BEFORE-i).add_prefix(f'{MINUTES_BEFORE-i}mins_ago_')], axis=1)
    

# delete rows with nans created by shifts
final_df = final_df.iloc[MINUTES_BEFORE:, :]


final_df = final_df.join(temp_zuz['temp_zuz'])

final_df = final_df.dropna(axis=0)
# dodanie temperatury żużla
final_df = final_df.rename(columns={'temp_zuz': 'TEMP_ZUZ'})


In [148]:
train, val, test = split(final_df)

X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model.predict(X_val))

33.44318245457594